# Analysis

In [ ]:
from file import read, write
import pandas as pd

In [ ]:
TEAMS = {}

for team in read('data/teams.json'):
    teamID = team["team_id"]
    TEAMS[teamID] = team

In [ ]:
games = read('data/games.json')

gamesToRemove = []
for i in range(len(games)):
    game = games[i]

    if 'start_time' not in game:
        gamesToRemove.append(i)

    remove = []
    for key, value in game.items():
        if type(value) == type("string"):
            remove.append(key)
    
    for key in remove:
        del game[key]

for index in sorted(gamesToRemove, reverse=True):
    del games[index]

In [ ]:
df = pd.DataFrame(games)

# clean
df = df.dropna(axis='columns')

# normalize
for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()     
    
df

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5, whiten=False)
pca.fit(df)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

In [ ]:
gameCounts = {}
for game in read('data/games.json'):
    red = game['red_id'] if 'red_id' in game else ''
    blue = game['blue_id'] if 'blue_id' in game else ''

    if red:
        if red in gameCounts:
            gameCounts[red] += 1 
        else: gameCounts[red] = 1
    if blue:
        if blue in gameCounts:
            gameCounts[blue] += 1
        else: gameCounts[blue] = 1

counts = []
for team, count in gameCounts.items():
    counts.append({'team_id': team, 'count': count})

counts.sort(key=lambda x: x['count'], reverse=True)

df = pd.DataFrame(counts)
print(df['count'].mean())
counts

In [ ]:
from pyvis.network import Network


teams = read('data/teams.json')

net = Network(notebook=True, cdn_resources='in_line')
for team in teams:
    net.add_node(team['team_id'], label=team['name'])

games = read('data/games.json')
for game in games:
    red = game['red_id']
    blue = game['blue_id']

    if red in TEAMS and blue in TEAMS:
        net.add_edge(red, blue, value=1)

net.show("network.html")